In [ ]:
from e3nn import o3
import torch

In [ ]:
# Create a symmetric 3x3 matrix
M = torch.randn(3, 3)
M = M + M.transpose(-1, -2)
M = M / M.norm()
print(M)

In [ ]:
irrep = o3.Irreps("1x0e + 1x2e")
irrep.dim

In [ ]:
# Analyse the tensor products
irreps_in1 = o3.Irreps("1x1o + 1x1o")
irreps_in2 = o3.Irreps("1x0e + 1x1o")
irreps_out = o3.Irreps("1x0e + 1x1o")

print(f"Dimension of the input1: {irreps_in1.dim}")
print(f"Dimension of the input2: {irreps_in2.dim}")
print(f"Dimension of the output: {irreps_out.dim}")

tp = o3.FullyConnectedTensorProduct(irreps_in1, irreps_in2, irreps_out)
x1 = torch.tensor([1.0, 0.0, 0.0,  2.0, 1.0, 0.0])
x2 = torch.tensor([123.4,          0.0, 1.0, 5.0])
out = tp(x1, x2)
out

In [ ]:
irreps_in1 = o3.Irreps("1x0e + 1x2e")
irreps_in2 = o3.Irreps("1x0e + 1x2e")
# irreps_out = o3.Irreps("1x0e + 1x1o")

print(f"Dimension of the input1: {irreps_in1.dim}")
print(f"Dimension of the input2: {irreps_in2.dim}")
# print(f"Dimension of the output: {irreps_out.dim}")

tp = o3.FullTensorProduct(irreps_in1, irreps_in2) 
tp.visualize()
x1 = torch.tensor([1.0,    0.0, 0.0, 2.0, 1.0, 0.0])
x2 = torch.tensor([1.4,    1.0, 2.0, 0.0, 1.0, 5.0])
out = tp(x1, x2)
out

In [ ]:
# Compare the types of tensor products
irrep_in = "1x2o"
o3_type = o3.Irreps(irrep_in)
print(f"Dimension of the input: {o3_type.dim}")

x = torch.tensor([1.0, 0.0, 0.0, 1.0, 2.0])
y = torch.tensor([0.0, 1.0, 0.0, 2.0, 1.0])

# Full Tensor Product
print("Full Tensor Product")
ftp = o3.FullTensorProduct(o3_type, o3_type)
out = ftp(x, y)
print(f"Dimension of the output: {out.shape}")
print(f"Output irreps: {ftp.irreps_out}")

print("Reduced Tensor Product")
rtp = o3.ReducedTensorProducts("ij=ji", i=irrep_in)
out = rtp(x, y)
print(f"Dimension of the output: {out.shape}")
print(f"Output irreps: {rtp.irreps_out}")